<a href="https://colab.research.google.com/github/Eshan-K-Kaushal/Voice_Chatbot/blob/main/T5_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt


# from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
from pprint import pprint

import sys
import os
import glob

!pip install simplet5
from simplet5 import SimpleT5

In [4]:
PATH = '/content/Combined_Dataset_Test.csv'
dataset = pd.read_csv(PATH)
pd.set_option('display.max_colwidth', None)
#dataset = dataset[['Question', 'Answer']]
dataset = dataset.sample(frac=1, random_state=1).reset_index()
#print(dataset.head())

q_list = "question: " + dataset['Question']                          # questions list to feed the model
n_list = dataset['Answer'] + " </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
#print(df.head())

# print(df['source_text'][1])
# print('\n')
# print(df['target_text'][1])

# splitting data into train and test data
train_data, val_data = train_test_split(df[:77], test_size=0.2)
test_data = df[70:77]
print(train_data.shape, val_data.shape, test_data.shape)


(60, 2) (16, 2) (6, 2)


In [5]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data,
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=10, use_gpu=True)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
model.load_model('/content/outputs/simplet5-epoch-9-train-loss-1.1736-val-loss-2.1116')

In [ ]:
test_data['source_text']

In [11]:
q_test = test_data['source_text'][71]
q_ans = test_data['target_text'][71]

print("Question: ", q_test)
print('-'*50)
print("Answer: ",q_ans)

Question:  question: I am a multi-objective solver. Ask me questions from any domain and I will answer them by breaking them down into steps and answering them step by step. Append "Let's solve it step by step" after every question Q: <'Question asked'>. Using the Question asked, find the domain of the question and generate a background document on the domain. Using the domain's background document answer the question step by step.
Is semi-closed endarterectomy of the superficial femoral artery combined with a short venous bypass in case of insufficient venous material an acceptable alternative for limb-threatening ischemia?
--------------------------------------------------
Answer:  Let's solve it step by step.

The aim of this study was to analyse the results of infragenual arterial revascularisation using semiclosed endarterectomy of the superficial femoral artery combined with a short venous bypass in patients with critical leg ischemia and insufficient venous material for a straig

In [12]:
predicted_ans = model.predict(q_test)[0]

print(predicted_ans)

Let's solve it step by step. Semiclosed endarterectomy of the superficial femoral artery combined with a short venous bypass in case of insufficient venous material is not considered an acceptable alternative for limb-threatening ischemia. The aim of this retrospective study was to evaluate whether semiclosed endartartectomy combined with short venous bypass in case of insufficient venous material is safe and effective.


In [13]:
y_test = [a for a in test_data['target_text'] ]

# get the predicted answers
y_pred = []
for q in test_data['source_text']:
    y_pred.append(model.predict(q)[0])


In [ ]:
y_pred

In [ ]:
!pip install rouge_score

In [16]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [17]:
# get the rouge score between test and predicted data
precision_rouge_scores, recall_rouge_scores = [], []
for i in range(len(y_pred)):
    precision_rouge_scores.append(round(scorer.score(y_test[i], y_pred[i])['rouge1'][0],2))
    recall_rouge_scores.append(round(scorer.score(y_test[i], y_pred[i])['rouge1'][1],2))

In [18]:
print(f'The average precision rouge scores of q&a for test data is: {round(np.mean(precision_rouge_scores),2)}')
print(f'The average recall rouge scores of q&a for test data is: {round(np.mean(recall_rouge_scores),2)}')

The average precision rouge scores of q&a for test data is: 0.75
The average recall rouge scores of q&a for test data is: 0.25
